# Import

In [1]:
import pandas as pd
import gtfs_kit as gk
import random
import matplotlib.pyplot as plt
import mglearn 

import helper #homemade function file helper.py
import warnings
import numpy as np
#from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder


# Code

In [2]:
n = 168605
s = 80000 #desired sample size
skip = sorted(random.sample(range(1,n),n-s))
main_dataset = pd.read_csv("../processed_files/main_dataset.csv", index_col=[0], skiprows=skip)
#main_dataset = pd.read_csv("../processed_files/main_dataset.csv", index_col=[0])

In [3]:
#Drop Columns
main_dataset = main_dataset.drop(['trip_id','codice_stazione','shape_dist_traveled','codice',
                                  'data_giorno','stop_lat','stop_lon','orario_arrivo','orario_partenza',
                                  'orario','ritardo_partenza','app_temp','rain','snow_fall','snow_depth','wind_speed','arrival_time',
                                  'departure_time','stop_sequence'
                                 ], axis=1)

#Round Columns
main_dataset = main_dataset.round({'temperature':0,'speed':0}) 

#Reset Index
main_dataset.reset_index(drop=True)


,time_diff,speed,dist_diff,direction_id,stop_code,ritardo_arrivo,temperature
0,0.00,0.0,0.00,0,6404,0,11.0
1,0.12,45.0,5.23,1,6404,3,11.0
2,0.00,0.0,0.00,0,6404,0,11.0
3,0.00,0.0,0.00,0,6404,0,11.0
4,0.12,45.0,5.23,1,6404,1,11.0
...,...,...,...,...,...,...,...
203386,0.13,65.0,8.70,1,6222,2,17.0
203387,0.13,65.0,8.70,1,6222,59,18.0
203388,0.13,65.0,8.70,1,6222,2,19.0
203389,0.13,65.0,8.70,1,6222,2,18.0


In [4]:
main_dataset.corr()


,time_diff,speed,dist_diff,direction_id,stop_code,ritardo_arrivo,temperature
time_diff,1.000000,0.376709,0.875297,0.118005,-0.019592,0.018972,0.021061
speed,0.376709,1.000000,0.630840,0.100955,-0.346468,0.190861,-0.009166
dist_diff,0.875297,0.630840,1.000000,0.116803,-0.104642,0.043679,0.030181
direction_id,0.118005,0.100955,0.116803,1.000000,-0.388928,0.006621,-0.026088
stop_code,-0.019592,-0.346468,-0.104642,-0.388928,1.000000,-0.146669,0.202230
ritardo_arrivo,0.018972,0.190861,0.043679,0.006621,-0.146669,1.000000,-0.027134
temperature,0.021061,-0.009166,0.030181,-0.026088,0.202230,-0.027134,1.000000


In [5]:
main_dataset.describe()


,time_diff,speed,dist_diff,direction_id,stop_code,ritardo_arrivo,temperature
count,203391.000000,203391.000000,203391.000000,203391.000000,203391.000000,203391.000000,203391.000000
mean,0.085813,49.072078,4.540367,0.319655,6336.067265,2.968652,15.484176
std,0.045993,19.586752,2.720783,0.466344,81.680520,4.409487,7.335810
min,0.000000,0.000000,0.000000,0.000000,6222.000000,0.000000,-7.000000
25%,0.070000,40.000000,2.620000,0.000000,6303.000000,0.000000,10.000000
50%,0.080000,51.000000,3.850000,0.000000,6312.000000,2.000000,14.000000
75%,0.120000,60.000000,6.340000,1.000000,6402.000000,4.000000,21.000000
max,0.430000,96.000000,20.910000,1.000000,6501.000000,187.000000,39.000000


# Machine Learning


In [6]:
X = np.column_stack((main_dataset.time_diff.values,main_dataset.speed.values, main_dataset.dist_diff.values, 
                     main_dataset.stop_code.values,main_dataset.temperature.values)) 
Y = main_dataset.ritardo_arrivo.values
# Constructing the input
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 20)
model = MLPRegressor(activation='relu',random_state = 20, alpha=0.001, early_stopping = False)
        
model.fit(X_train, y_train)
pred = model.predict(X_test)
test_set_rsquared = model.score(X_test,y_test)
test_set_rmse = np.sqrt(mean_squared_error(y_test,pred))
print(test_set_rsquared)
print(test_set_rmse)

-0.7055125437144687
5.716487868377302
